In [232]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from category_encoders import OneHotEncoder
import warnings
import random
import os
plt.style.use('ggplot')
warnings.filterwarnings(action='ignore')

In [233]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [275]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [235]:
# 124-127

In [276]:
def cyclical_feature(df): 
    df['sin_time'] = np.sin(2*np.pi*df.HOUR/24)
    df['cos_time'] = np.cos(2*np.pi*df.HOUR/24)
def timestamp_split(df):
    df['YEAR'] = df['TIMESTAMP'].str.split('-').str[0].astype('int')
    df['MONTH'] = df['TIMESTAMP'].str.split('-').str[1].astype('int')
    df['DAY'] = df['TIMESTAMP'].str.split('-').str[2]
    df['DAY'] = df['DAY'].str.split().str[0].astype('int')
    df['TIME'] = df['TIMESTAMP'].str.split().str[1]
    df['HOUR'] = df['TIME'].str.split(':').str[0].astype('int')
    df['TIME_MIN'] = df['TIME'].str.split(':').str[1].astype('int')+df['HOUR']*60

In [278]:
timestamp_split(train)
cyclical_feature(train)

In [248]:
del_col = []
for col in train.describe().columns:
    if train[col].std() >= 1000 :
        print(col)
        print("range : ",train[col].max()-train[col].min())
        print("std : ",train[col].std())
        del_col.append(col)
        print("-------")

X_123
range :  52730.0
std :  15036.418890416313
-------
X_124
range :  37683.0
std :  10750.383291637096
-------
X_125
range :  39477.0
std :  11307.951227059795
-------
X_126
range :  34052.0
std :  10155.391143601983
-------
X_127
range :  37847.0
std :  11063.650413933403
-------
X_128
range :  25759.0
std :  8290.625484239017
-------
X_129
range :  25759.0
std :  8233.16089710643
-------
X_130
range :  25349.0
std :  9234.859280734025
-------
X_131
range :  25349.0
std :  9234.859280734025
-------
X_370
range :  121533.0
std :  41455.62393833486
-------
X_371
range :  39212.0
std :  10875.204157941778
-------
X_372
range :  39212.0
std :  10628.805641028688
-------
X_373
range :  39158.0
std :  12506.636876956556
-------
X_374
range :  39158.0
std :  12506.636876956556
-------
X_978
range :  65480.0
std :  32598.08650522215
-------
X_979
range :  44341.39617
std :  16372.72617958863
-------
X_991
range :  30000.0
std :  12838.30278302486
-------
X_996
range :  237848.0
std :  6645

In [249]:
train = train.drop(columns=del_col)
test = test.drop(columns=del_col)

In [250]:
col_list = train.columns
nan_list = []
nan_cnt = []
nan_col = []
full_list = []
for col in col_list:
    if train[col].isnull().sum() == 0 :
        full_list.append(col)
        continue
    nan_list.append([col, train[col].isnull().sum()])
    nan_cnt.append(train[col].isnull().sum())
    nan_col.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_list :
    if col[1] == 598 :
        del_col.append(col[0])
train = train.drop(columns=del_col)
test = test.drop(columns=del_col)
train.head(3)

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,NaN


In [251]:
trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

In [252]:
col_list = train.columns
nan_listA_31 = []
nan_cntA_31 = []
nan_colA_31 = []
full_listA_31 = []
for col in col_list:
    if trainA_31[col].isnull().sum() == 0 :
        full_listA_31.append(col)
        continue
    nan_listA_31.append([col, trainA_31[col].isnull().sum()])
    nan_cntA_31.append(trainA_31[col].isnull().sum())
    nan_colA_31.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listA_31 :
    if col[1] == len(trainA_31) :
        del_col.append(col[0])
trainA_31 = trainA_31.drop(columns=del_col)
testA_31 = testA_31.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = trainA_31.columns
for col in col_list[6:] :
    if trainA_31[col].nunique()==1 :
        del_col.append(col)
trainA_31 = trainA_31.drop(columns=del_col)
testA_31 = testA_31.drop(columns=del_col)

In [253]:
col_list = train.columns
nan_listO = []
nan_cntO = []
nan_colO = []
full_listO = []
for col in col_list:
    if train_O_31[col].isnull().sum() == 0 :
        full_listO.append(col)
        continue
    nan_listO.append([col, train_O_31[col].isnull().sum()])
    nan_cntO.append(train_O_31[col].isnull().sum())
    nan_colO.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listO :
    if col[1] == len(train_O_31) :
        del_col.append(col[0])
train_O_31 = train_O_31.drop(columns=del_col)
test_O_31 = test_O_31.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = train_O_31.columns
for col in col_list[6:] :
    if train_O_31[col].nunique()==1 :
        del_col.append(col)
train_O_31 = train_O_31.drop(columns=del_col)
test_O_31 = test_O_31.drop(columns=del_col)

In [254]:
col_list = train.columns
nan_listT = []
nan_cntT = []
nan_colT = []
full_listT = []
for col in col_list:
    if train_T_31[col].isnull().sum() == 0 :
        full_listT.append(col)
        continue
    nan_listT.append([col, train_T_31[col].isnull().sum()])
    nan_cntT.append(train_T_31[col].isnull().sum())
    nan_colT.append(col)
    
'''모든값이 결측값이면 제거'''
del_col = []
for col in nan_listT :
    if col[1] == len(train_T_31) :
        del_col.append(col[0])
train_T_31 = train_T_31.drop(columns=del_col)
test_T_31 = test_T_31.drop(columns=del_col)

'''값이 1개 존재하면 제거'''
del_col = []
col_list = train_T_31.columns
for col in col_list[6:] :
    if train_T_31[col].nunique()==1 :
        del_col.append(col)
train_T_31 = train_T_31.drop(columns=del_col)
test_T_31 = test_T_31.drop(columns=del_col)

In [255]:
trainA_31_x = trainA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
testA_31_x = testA_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
train_T_31_x = train_T_31.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality','PRODUCT_CODE'])
test_T_31_x = test_T_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
train_O_31_x = train_O_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
test_O_31_x = test_O_31.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])

# classification
trainA_31_y_c = trainA_31['Y_Class']
train_T_31_y_c = train_T_31['Y_Class']
train_O_31_y_c = train_O_31['Y_Class']

# regression
trainA_31_y_r = trainA_31['Y_Quality']
train_T_31_y_r = train_T_31['Y_Quality']
train_O_31_y_r = train_O_31['Y_Quality']

In [256]:
print(trainA_31_x.shape,'\n',testA_31_x.shape,'\n',train_T_31_x.shape,'\n',test_T_31_x.shape,'\n',train_O_31_x.shape,'\n',test_O_31_x.shape)

(249, 1786) 
 (67, 1786) 
 (343, 545) 
 (239, 545) 
 (6, 494) 
 (4, 494)


In [258]:
trainA_31_x=trainA_31_x.fillna(-1)
testA_31_x=testA_31_x.fillna(-1)
train_T_31_x=train_T_31_x.fillna(-1)
test_T_31_x=test_T_31_x.fillna(-1)
train_O_31_x=train_O_31_x.fillna(-1)
test_O_31_x=test_O_31_x.fillna(-1)

In [259]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(trainA_31_x[i])
    trainA_31_x[i] = le.transform(trainA_31_x[i])
    
    for label in np.unique(testA_31_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    testA_31_x[i] = le.transform(testA_31_x[i]) 
print('Done.')

Done.


In [260]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_T_31_x[i])
    train_T_31_x[i] = le.transform(train_T_31_x[i])
    
    for label in np.unique(test_T_31_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_T_31_x[i] = le.transform(test_T_31_x[i]) 
print('Done.')

Done.


In [261]:
# qualitative to quantitative
qual_col = ['LINE']
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_O_31_x[i])
    train_O_31_x[i] = le.transform(train_O_31_x[i])
    
    for label in np.unique(test_O_31_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_O_31_x[i] = le.transform(test_O_31_x[i]) 
print('Done.')

Done.


In [262]:
#from imblearn.combine import SMOTETomek

In [263]:
from catboost import *

In [264]:
model = CatBoostRegressor(random_state=1234,verbose=500,iterations=2000,learning_rate=0.033)
model.fit(trainA_31_x, trainA_31_y_r)
pred_a = model.predict(testA_31_x)

0:	learn: 0.0098641	total: 55.1ms	remaining: 1m 50s
500:	learn: 0.0006188	total: 24.1s	remaining: 1m 12s
1000:	learn: 0.0000581	total: 51.5s	remaining: 51.4s
1500:	learn: 0.0000055	total: 1m 22s	remaining: 27.3s
1999:	learn: 0.0000006	total: 1m 53s	remaining: 0us


In [265]:
#model = CatBoostRegressor(random_state=110,verbose=500,iterations=500)
model.fit(train_T_31_x, train_T_31_y_r)
pred_t = model.predict(test_T_31_x)

0:	learn: 0.0047201	total: 21.8ms	remaining: 43.5s
500:	learn: 0.0005142	total: 12.8s	remaining: 38.2s
1000:	learn: 0.0000543	total: 26.4s	remaining: 26.3s
1500:	learn: 0.0000079	total: 38.1s	remaining: 12.7s
1999:	learn: 0.0000010	total: 50.5s	remaining: 0us


In [266]:
#model = CatBoostRegressor(random_state=110,verbose=500,iterations=500)
model.fit(train_O_31_x, train_O_31_y_r)
pred_o = model.predict(test_O_31_x)

0:	learn: 0.0031730	total: 5.78ms	remaining: 11.6s
500:	learn: 0.0000474	total: 2.11s	remaining: 6.31s
1000:	learn: 0.0000008	total: 3.98s	remaining: 3.97s
1500:	learn: 0.0000000	total: 5.81s	remaining: 1.93s
1999:	learn: 0.0000000	total: 8.2s	remaining: 0us


In [267]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

In [268]:
testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

In [269]:
testA_31.loc[(testA_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

test_T_31.loc[(test_T_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

test_O_31.loc[(test_O_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

In [270]:
submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

In [271]:
submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

In [272]:
pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('2000.csv',index=False)

In [211]:
submito

,PRODUCT_ID,Y_Class
0,TEST_138,1
1,TEST_256,1
2,TEST_257,1
3,TEST_287,1


In [157]:
pd.read_csv('catboost.csv')[pd.read_csv('catboost.csv')['PRODUCT_ID']=='TEST_257']

,PRODUCT_ID,Y_Class
257,TEST_257,1


In [17]:
model = CatBoostRegressor(random_state=110,verbose=500,iterations=500)
model.fit(trainA_31_x, trainA_31_y_r)
pred_a = model.predict(testA_31_x)

Learning rate set to 0.051607
0:	learn: 0.0097830	total: 221ms	remaining: 1m 50s
499:	learn: 0.0001949	total: 31.4s	remaining: 0us


In [18]:
model.fit(train_T_O_x, train_T_O_y_r)
pred_t = model.predict(test_T_O_x)

Learning rate set to 0.054803
0:	learn: 0.0046878	total: 31.2ms	remaining: 15.6s
499:	learn: 0.0001367	total: 12.5s	remaining: 0us


In [19]:
testA_31['Y_quanlity'] = pred_a
test_T_O['Y_quanlity'] = pred_t

In [20]:
testA_31['Y_Class'] = 1
test_T_O['Y_Class'] = 1

In [21]:
testA_31.loc[(testA_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>0.5349), 'Y_Class'] = 2

test_T_O.loc[(test_T_O['Y_quanlity']<0.52507), 'Y_Class'] = 0
test_T_O.loc[(test_T_O['Y_quanlity']>0.5349), 'Y_Class'] = 2

In [22]:
submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')

In [23]:
submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_O[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt]).sort_values(by='PRODUCT_ID').to_csv('sub0203_cat_reg_time_del.csv',index=False)

In [212]:
testA_31['Y_quanlity'] = pred_a
testA_31['Y_Class'] = 1
testA_31.loc[(testA_31['Y_quanlity']<0.52507), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>0.5349), 'Y_Class'] = 2
testA_31['Y_Class'].values

array([0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1], dtype=int64)

In [122]:
testA_31['Y_Class'].values

array([0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0], dtype=int64)

In [93]:
testA_31['Y_Class'].values

array([0, 1, 1, 1, 0, 0, 1, 1, 1, 2, 0, 0, 0, 1, 0, 2, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0], dtype=int64)

In [24]:
testA_31['Y_Class'].values

array([0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0], dtype=int64)

In [123]:
test_T_31['Y_Class'].values

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

In [94]:
test_T_31['Y_Class'].values

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

In [35]:
test_T_O['Y_Class'].values

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1], dtype=int64)

In [95]:
test_O_31['Y_Class'].values

array([1, 1, 1, 1], dtype=int64)

In [25]:
test_T_O['Y_Class'].values

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1], dtype=int64)